![Banner ETL](<../5. Sources/Images/banner_etl.gif>)
# <h2 align=center>**`ETL GOOGLE MAPS (REVIEW-ESTADOS)`**</h2>
Debido a que existen 51 carpetas (1 para cada estado de USA) con varios archivos .json cada una, se debe buscar la ruta de acceso en el ordenador donde se encuentran las carpetas y sus archivos. Luego, un ciclo 'for' alimentará una función con la ruta de cada carpeta y esta (la función) unirá cada archivo .json en un dataframe. Cada uno de estos dataframes se cruzará con los datos contenidos en el dataframe 'metada' dando como resultado solo información (reviews o reseñas) del rubro 'hotel'. Al final, se generará un solo dataframe con toda la información. 

Importar las librerias necesarias

In [2]:
import pandas as pd
import numpy as np
import os

Importar el dataset 'metadata' para su posterior cruce con el dataframe de cada estado

In [8]:
#Dataframe 'metadata'
df_metadata = pd.read_csv('D:\\Aldemar\\cursos\\SoyHenry\\Cohorte DataPT02\\PF_DS\\Google Maps\\metadata-sitios\\df_metadata.csv')
df_metadata.head(2)

,Unnamed: 0,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,category_hotel
0,175,Basecamp Guesthouse,"Basecamp Guesthouse, 49010 SE Middle Fork Rd, ...",0x54907fca2751e187:0x3d6ae31ac18da483,NaN,47.471332,-121.686928,['Hotel'],5.0,8,NaN,NaN,NaN,NaN,"['0x54907c35e89f333f:0x121a0cbfbbb39d3a', '0x5...",https://www.google.com/maps/place//data=!4m2!3...,Hotel
1,3611,Sugar River Loft,"Sugar River Loft, 929 W Exchange St, Brodhead,...",0x88062b28cb53ef6d:0x94df72568fddf391,NaN,42.619862,-89.377890,['Hotel'],4.7,8,NaN,NaN,NaN,NaN,"['0x88062b123701348f:0x5c12c6134b41325', '0x88...",https://www.google.com/maps/place//data=!4m2!3...,Hotel


Crear función que retorna un dataframe para cada estado

In [15]:
def union_arch_json(estado:str): #creación de la función

    df_estado = pd.DataFrame() #dataframe vacío (será el que contenga los datos de cada estado)

    directorio_estado = f'D:\\Aldemar\\cursos\\SoyHenry\\Cohorte DataPT02\\PF_DS\Google Maps\\reviews-estados\\reviews-estados\\{estado}' #ruta de los archivos .json para cada estado
    archivos_estado = os.listdir(directorio_estado) #lista de los archivos .json

    #crear dataframe con el contenido de los archivos .json
    for archivo in archivos_estado:
        df_archivo = pd.read_json(f'reviews-estados\\{estado}\\{archivo}', lines=True)
        df_archivo['estado'] = estado.replace('review-','') #creación de la columna 'estado'

        #concatenación de los datos en el dataframe
        if df_estado.shape[0] == 0:
            df_estado = df_archivo
        else:
            df_estado = pd.concat([df_estado, df_archivo])
        
    return df_estado #retorno del dataframe

Cruzar cada dataframe generado anteriormente con el dataframe metadata para posterior concatenación en un solo dataframe

In [16]:
directorio_estados = 'D:\\Aldemar\\cursos\\SoyHenry\\Cohorte DataPT02\\PF_DS\Google Maps\\reviews-estados\\reviews-estados' #ruta de las 51 carpetas (una para cada estado)
lista_estados = os.listdir(directorio_estados) #lista de las 51 carpetas 

df_estados = pd.DataFrame() #dataframe vacío (será el que contenga los datos del cruce)

#Alimentar la función, cruzar los dataframes 'metadata' y 'df_estado' y crear un solo dataframe que disponibilioce los datos
for estado in lista_estados:
    df_estado = union_arch_json(estado)
    df_estado = pd.merge(df_metadata, df_estado, on='gmap_id', how='inner')

    #unión de cada dataframe cruzado
    if df_estados.shape[0] == 0:
        df_estados = df_estado
    else:
        df_estados = pd.concat([df_estados, df_estado])

df_estados.to_csv("df_estados.csv") #exportar el dataframe en un archivo .csv

Visualización del dataframe final

In [17]:
df_estados

,Unnamed: 0,name_x,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,category_hotel,user_id,name_y,time,rating,text,pics,resp,estado
0,71118,Holiday Inn Express & Suites Fultondale,"Holiday Inn Express & Suites Fultondale, 1733 ...",0x88890491e9416625:0x2f85307a06ddb16,Unfussy rooms & suites in a relaxed hotel offe...,33.609814,-86.807733,['Hotel'],3.7,38,NaN,NaN,NaN,NaN,"['0x88890492ef4ca701:0x34f0c46c887eaf73', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Hotel,1.126652e+20,Charlie Cline,1454821094934,4,Good location. Nice staff. Clean. Breakfast ar...,None,None,Alabama
1,71118,Holiday Inn Express & Suites Fultondale,"Holiday Inn Express & Suites Fultondale, 1733 ...",0x88890491e9416625:0x2f85307a06ddb16,Unfussy rooms & suites in a relaxed hotel offe...,33.609814,-86.807733,['Hotel'],3.7,38,NaN,NaN,NaN,NaN,"['0x88890492ef4ca701:0x34f0c46c887eaf73', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Hotel,1.086380e+20,Jennifer Passmore,1491966256419,5,Super clean very friendly,None,None,Alabama
2,71118,Holiday Inn Express & Suites Fultondale,"Holiday Inn Express & Suites Fultondale, 1733 ...",0x88890491e9416625:0x2f85307a06ddb16,Unfussy rooms & suites in a relaxed hotel offe...,33.609814,-86.807733,['Hotel'],3.7,38,NaN,NaN,NaN,NaN,"['0x88890492ef4ca701:0x34f0c46c887eaf73', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Hotel,1.090791e+20,TheBeckyboo926,1476494074021,5,Amazing! We had a conference room and it was a...,None,None,Alabama
3,71118,Holiday Inn Express & Suites Fultondale,"Holiday Inn Express & Suites Fultondale, 1733 ...",0x88890491e9416625:0x2f85307a06ddb16,Unfussy rooms & suites in a relaxed hotel offe...,33.609814,-86.807733,['Hotel'],3.7,38,NaN,NaN,NaN,NaN,"['0x88890492ef4ca701:0x34f0c46c887eaf73', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Hotel,1.170887e+20,yolanda arellano,1479132766153,3,A bit on the pricey side but they take on pets...,None,None,Alabama
4,71118,Holiday Inn Express & Suites Fultondale,"Holiday Inn Express & Suites Fultondale, 1733 ...",0x88890491e9416625:0x2f85307a06ddb16,Unfussy rooms & suites in a relaxed hotel offe...,33.609814,-86.807733,['Hotel'],3.7,38,NaN,NaN,NaN,NaN,"['0x88890492ef4ca701:0x34f0c46c887eaf73', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Hotel,1.170284e+20,Greg Randall,1464460773963,4,Solid hotel. Staff was friendly. Room was clean.,None,None,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,62314,Lariat Motel,"Lariat Motel, 600 Central Ave, Cheyenne, WY 82007",0x876f3ad1b5c6bc99:0x849e85633b455621,NaN,41.124572,-104.808938,"['Motel', 'Hotel']",2.0,38,NaN,NaN,NaN,NaN,"['0x876f3ab8b9a67121:0xcc7e3b037547ebbe', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Hotel,1.100741e+20,Michael Daniel,1593481758525,3,None,None,None,Wyoming
56,62314,Lariat Motel,"Lariat Motel, 600 Central Ave, Cheyenne, WY 82007",0x876f3ad1b5c6bc99:0x849e85633b455621,NaN,41.124572,-104.808938,"['Motel', 'Hotel']",2.0,38,NaN,NaN,NaN,NaN,"['0x876f3ab8b9a67121:0xcc7e3b037547ebbe', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Hotel,1.124768e+20,Kerri Clem,1593917456458,2,None,None,None,Wyoming
57,62314,Lariat Motel,"Lariat Motel, 600 Central Ave, Cheyenne, WY 82007",0x876f3ad1b5c6bc99:0x849e85633b455621,NaN,41.124572,-104.808938,"['Motel', 'Hotel']",2.0,38,NaN,NaN,NaN,NaN,"['0x876f3ab8b9a67121:0xcc7e3b037547ebbe', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Hotel,1.100101e+20,Gina Carabajal,1568741311826,1,None,None,None,Wyoming
58,62314,Lariat Motel,"Lariat Motel, 600 Central Ave, Cheyenne, WY 82007",0x876f3ad1b5c6bc99:0x849e85633b455621,NaN,41.124572,-104.808938,"['Motel', 'Hotel']",2.0,38,NaN,NaN,NaN,NaN,"['0x876f3ab8b9a67121:0xcc7e3b037547ebbe', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,Hotel,1.006094e+20,Adam,1505881621069,2,None,None,None